### IO and utilities

In [2]:
import os
from dataclasses import dataclass, fields
import re

def read_file(path: str) -> list[str]:
    with open(os.path.join(os.getcwd(), path), "r") as f:
        return [line.strip() for line in f.readlines()]


# Day 1: Trebuchet?!


In [3]:
def getnumber(line: str) -> int:
    numbers = [c for c in line if c.isdigit()]
    return int(numbers[0] + numbers[-1])

day1_1 = read_file("day1/example.txt")
print(sum([getnumber(l) for l in day1_1]))


142


Part 2, search each line for the first digit or word, the search from back after first digit or word.

In [4]:
def test(substr: str) -> str:
    numbers = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9'
    }

    for word, digit in numbers.items():
        if word in substr:
            return digit

    return ''

def find_first_digit(inline: str, reverse = False) -> str:
    buffer = ''
    line = inline[::-1] if reverse else inline

    for c in line:
        if c.isdigit():
            return c

        if reverse:
            buffer = c + buffer
        else:
            buffer = buffer + c

        candidate = test(buffer)
        if candidate:
            return candidate

def replace_text(line: str) -> str:
    return find_first_digit(line) + find_first_digit(line, True)
   
day1_2 = read_file("day1/example.txt")
parsed = [int(replace_text(l)) for l in day1_2]
print(sum(parsed))

142


# Day 2: Cube Conundrum

### Data representation and parsing

In [5]:
@dataclass
class GameSet():
    red: int
    green: int
    blue: int

@dataclass
class Game():
    id: int
    sets: list[GameSet]

def parse_set(s: str) -> GameSet:
    gameset = GameSet(0,0,0)
    cubesstr = s.split(',')

    for cube in cubesstr:
        n = int(re.search('\d+', cube).group())
        if 'red' in cube:
            gameset.red = n
        elif 'green' in cube:
            gameset.green = n
        else:
            gameset.blue = n
    
    return gameset


def parse(line: str) -> Game:
    g = Game(-1, [])
    
    gamesrt, setsstr = line.split(':')
    g.id = int(gamesrt[4:])
    g.sets = [parse_set(s) for s in setsstr.split(';')]
    return g

day2_1 = read_file('day2/example.txt')
games = [parse(l) for l in day2_1]

### Part 1: Finding possible games

In [6]:

def max_set(a: GameSet, b: GameSet) -> GameSet:
    return GameSet(max(a.red, b.red), max(a.green, b.green), max(a.blue, b.blue))

def possible(game: Game, max_allowed: GameSet) -> bool:
    game_max = GameSet(0,0,0)

    for gameset in game.sets:
        game_max = max_set(game_max, gameset)

    return game_max.red <= max_allowed.red and game_max.green <= max_allowed.green and game_max.blue <= max_allowed.blue

valid_game_ids = [g.id for g in games if possible(g, GameSet(12, 13, 14))]
print(sum(valid_game_ids))


8


### Part 2: Finding mimimum possible cubes and summing power of these sets

In [7]:
def minimum_possible_cubes(game: Game) -> GameSet:
    game_min = GameSet(0,0,0)

    for gameset in game.sets:
        game_min = max_set(game_min, gameset)

    return game_min

def game_power(game: Game) -> int:
    s = minimum_possible_cubes(game)
    return s.red * s.green * s.blue

print(sum([game_power(g) for g in games]))

2286


# Day 3: Gear Ratios

### Utils

In [27]:

@dataclass
class Coord():
    row: int
    col: int


StringMatrix = list[str]

def getAdjecent(coord: Coord, total_rows: int, total_cols: int) -> list[Coord] :
    result = []
    
    neighbouring_left = coord.col > 0
    neighbouring_right = coord.col < total_cols
    neighbouring_up = coord.row > 0
    neighbouring_down = coord.row < total_rows

    if neighbouring_up:
        if neighbouring_left:
            result.append(Coord(coord.row - 1, coord.col - 1))
        
        result.append(Coord(coord.row - 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row - 1, coord.col + 1))
    
    if neighbouring_left:
        result.append(Coord(coord.row, coord.col - 1))

    if neighbouring_right:
        result.append(Coord(coord.row, coord.col + 1))

    if neighbouring_down:
        if neighbouring_left:
            result.append(Coord(coord.row + 1, coord.col - 1))
        
        result.append(Coord(coord.row + 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row + 1, coord.col + 1))

    return result

def get_char(data: StringMatrix, coord: Coord) -> str:
    return data[coord.row][coord.col]

def is_digit(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord).isdigit()

def getIntIndex(data: StringMatrix, coord: Coord) -> (int,int):
    found_first = False
    found_last = False 

    f_index = coord.col
    l_index = coord.col

    while not found_first:
        candidate = f_index - 1
        if candidate >= 0 and data[coord.row][candidate].isdigit():
            f_index = candidate
        else:
            found_first = True

    while not found_last:
        candidate = l_index + 1
        if candidate < len(data[coord.row]) and data[coord.row][candidate].isdigit():
            l_index = candidate
        else:
            found_last = True

    return f_index, l_index

def remove_data(data: StringMatrix, row, begin, end):
    current = data[row]
    data[row] = current[:begin] + 'x' * (end + 1 - begin) + current[end+1:]



### Part 1

In [28]:
def is_symbol(data: StringMatrix, coord: Coord) -> bool:
    symbols = ["+","*","#","$","%","@","#","&","=","-",'/']
    return get_char(data, coord) in symbols

result = 0
data = read_file("day3/example.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_symbol(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    result += number
                    remove_data(data, a.row, first, last)

print(result)

4361


### Part 2 - find gears

In [29]:
def is_gear(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord) == '*'

result = 0
data = read_file("day3/example.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_gear(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))
            
            numbers = []

            data_before = data

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    numbers.append(number)
                    remove_data(data, a.row, first, last)

            if len(numbers) == 2:
                result += numbers[0] * numbers[1]
            else: 
                # No gear! Restore data
                data = data_before
print(result)

467835


# Day 4: Scratchcards

### Part 1

In [11]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [])

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def worth(c: Card) -> int:
    wins = c.numbers.intersection(c.winning_numbers)
    return 0 if len(wins) == 0 else pow(2, len(wins) - 1)

data = read_file("day4/example.txt")
cards = [parse_card(l) for l in data]
winnings = [worth(c) for c in cards]
print(sum(winnings))


13


### Part 2

In [12]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    copies: int
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [], 1)

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def wins(c: Card) -> int:
    wins = len(c.numbers.intersection(c.winning_numbers))
    return range(c.id, c.id+wins)
      

data = read_file("day4/example.txt")
cards = [parse_card(l) for l in data]
for card in cards:
    new_copies = wins(card)
    for id in new_copies:
        if id < len(cards):
            cards[id].copies += card.copies

total = sum([c.copies for c in cards])
print(total)

30


# Day 5: If You Give A Seed A Fertilizer

### Utils - shared code between Part 1 and Part 2

In [13]:
import sys
import operator

@dataclass
class Table_item():
    destination_range_start: int
    source_range_start: int
    range_length: int

def within (src: int, entry: Table_item) -> int:
    src_delta = src - entry.source_range_start
    if src_delta >= 0 and src_delta < entry.range_length:
        return entry.destination_range_start + src_delta
    return -1

def parse_tables(lines: list[str]) -> dict[str, list[Table_item]]:
    tables = {}

    parsing_map = False    
    current_map = ''

    for line in lines:
        if len(line) == 0:
            parsing_map = False
            if current_map:
                current_map = ''
            continue
        if not parsing_map and ' map:' in line:
            current_map = line.split(" map:")[0]
            parsing_map = True
            continue

        if parsing_map:
            d, s, r = [int(x) for x in line.split()]
            
            if not current_map in tables:
                tables[current_map] = []
            
            tables[current_map].append(Table_item(d, s, r))
    
    for table in tables.values():
        table.sort(key=operator.attrgetter('source_range_start'))

    return tables

def populate_seeds(seed_ids: list[int], tables: dict[str, list[Table_item]]) -> list[list[int]]:
    seeds = [[id] + [-1] * 7 for id in seed_ids]
    keys = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    
    for seed in seeds:
        for i, key in enumerate(keys, 1):
            src = i-1
            dst = i
            
            seed[dst] = seed[src] # Default to prev value

            for entry in tables[key]:
                v = within(seed[src], entry)
                if v > -1:
                    seed[dst] = v
                    break
    
    return seeds

def find_lowest_location(seeds: list[list[int]]) -> int:
    lowest_location = sys.maxsize

    for seed in seeds:
        lowest_location = min(lowest_location, seed[7]) # Location at index 7

    return lowest_location


### Part 1

In [14]:
def parse_seeds(line: str) -> list[int]:
    return [int(x) for x in line[7:].split()]

data = read_file("day5/example.txt")
seed_ids = parse_seeds(data[0])
tables = parse_tables(data[1:])
seeds = populate_seeds(seed_ids, tables)
print(find_lowest_location(seeds))

35


### Part 2
Needed to solve it based on seed ranges instead of simulating every seed. The solution from part 1 when applied to part 2 was too slow

In [15]:
@dataclass
class Number_Range():
    start: int
    length: int

def parse_seeds(line: str) -> list[Number_Range]:
    seed_numbers = [int(x) for x in line[7:].split()]
    
    seeds = []
    for i in range(0, len(seed_numbers), 2):
        seeds.append(Number_Range(seed_numbers[i], seed_numbers[i+1]))

    seeds.sort(key=operator.attrgetter('start'))
    return seeds

def convert_range(range: Number_Range, entry: Table_item) -> (Number_Range | None, Number_Range | None, Number_Range | None):
    range_end = range.start + range.length
    entry_end = entry.source_range_start + entry.range_length

    if range_end < entry.source_range_start:
        below = range
        return below, None, None
    
    if entry_end <= range.start:
        above = range
        return None, None, above

    # all within
    if range.start >= entry.source_range_start and range_end <= entry_end:
        start_delta = range.start - entry.source_range_start
        converted = Number_Range(entry.destination_range_start + start_delta, range.length)
        return None, converted, None
    
    if range.start < entry.source_range_start and range_end <= entry_end:
        delta = entry.source_range_start - range.start
        below = Number_Range(range.start, delta)
        converted = Number_Range(entry.destination_range_start, range.length - delta)
        return below, converted, None

    if range.start >= entry.source_range_start and range_end > entry_end:
        delta = range_end - entry_end
        above = Number_Range(entry_end, delta)
        delta_start = range.start - entry.source_range_start
        converted = Number_Range(entry.destination_range_start + delta_start, range.length - delta)
        return None, converted, above

    if range.start < entry.source_range_start and range_end > entry_end:
        delta_b = entry.source_range_start - range.start
        below = Number_Range(range.start, delta_b)

        delta_a = range_end - entry_end
        above = Number_Range(entry_end, delta_a)
        
        converted = Number_Range(entry.destination_range_start, entry.range_length)
        return below, converted, above
    
def lookup(range: Number_Range, table: list[Table_item]) -> list[Number_Range]:
    # We assume that table is sorted based on source start

    remaining = range
    processed = []
    for entry in table:
        below, converted, above = convert_range(remaining, entry)

        if below:
            processed.append(below)

        if converted:
            processed.append(converted)
        
        remaining = above

        if not remaining:
            break

    if remaining:
        processed.append(remaining)
    
    return processed

def solve(seeds: list[Number_Range], tables: dict[str, list[Table_item]]) -> int:
    table_names = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    
    unprocessed_range = seeds
    proccessed = []

    for name in table_names:
        proccessed = []
        for nr in unprocessed_range:
            proccessed.extend(lookup(nr, tables[name]))

        unprocessed_range = proccessed
    
    min_location = sys.maxsize
    for x in proccessed:
        min_location = min(min_location, x.start)

    return min_location

data = read_file("day5/example.txt")
seed_ranges = parse_seeds(data[0])
tables = parse_tables(data[1:])

lowest_location = solve(seed_ranges, tables)
print(lowest_location)

46


# Day 6: Wait For It

### Shared code

In [16]:
import re
import math

@dataclass
class Record():
    time: int
    distance: int

def distance(holdtime: int, total: int) -> int:
    return holdtime * (total-holdtime)

def ways_to_beat(race: Record) -> int:
    wins = 0
    half = int((race.time + 1) * 0.5)
    first_beat = math.ceil(race.distance/race.time)
    for i in range(first_beat,half):
        if distance(i, race.time) > race.distance:
            wins += 2
    
    return wins + (race.time + 1) % 2


### Part 1

In [17]:
def line_to_numbers(line: str) -> list[int]:
    return [int(s) for s in re.findall("\D*(\d+)", line)]


def parse_races(lines: list[str]) -> list[Record]:
    times = line_to_numbers(lines[0])
    distances = line_to_numbers(lines[1])
    return [Record(time, distance) for time, distance in zip(times, distances)]

data = read_file("day6/example.txt")
races = parse_races(data)

math.prod([ways_to_beat(race) for race in races]) 

288

### Part 2

In [18]:
def line_to_number(line: str) -> int:
    return int("".join([s for s in re.findall("\D*(\d+)", line)]))


def parse_race(lines: list[str]) -> Record:
    time = line_to_number(lines[0])
    distance = line_to_number(lines[1])
    return Record(time, distance)

data = read_file("day6/example.txt")
race = parse_race(data)
wins = ways_to_beat(race)
print(wins)

71503


# Day 7: Camel Cards 

### Part 1

In [19]:
from enum import IntEnum
from functools import cmp_to_key

@dataclass
class Player():
    hand: str
    bid: int


class Card_type(IntEnum):
    HIGH_CARD = 1
    PAIR = 2
    TWO_PAIR = 3
    THREE_OF_A_KIND = 4
    FULL_HOUSE = 5
    FOUR_OF_A_KIND = 6
    FIVE_OF_A_KIND = 7

def parse_players(lines: list[str]) -> list[Player]:
    pattern = re.compile(r'(.{5})\s+(\d*)')
    players = []
    for line in lines:
        hand, bid = pattern.search(line).groups()
        players.append(Player(hand, int(bid)))

    return players

def get_type(cards: str) -> Card_type:
    unique_labels = set(cards)
    
    match len(unique_labels):
        case 1:
            return Card_type.FIVE_OF_A_KIND
        case 2:
            # Full house or Four of a kind
            occurence = cards.count(unique_labels.pop())
            if  occurence == 4 or occurence == 1:
                return Card_type.FOUR_OF_A_KIND
            return Card_type.FULL_HOUSE
        case 3:
            # Three of a kind or Two pair
            occurence = cards.count(unique_labels.pop())

            if occurence == 3:
                return Card_type.THREE_OF_A_KIND
            elif occurence == 2:
                return Card_type.TWO_PAIR
            else:
                occurence = cards.count(unique_labels.pop())
                if occurence == 2:
                    return Card_type.TWO_PAIR
                return Card_type.THREE_OF_A_KIND
        case 4:
            return Card_type.PAIR
        case other:
            return Card_type.HIGH_CARD

def card_value(card: str):
    match card:
        case 'A':
            return 14
        case 'K':
            return 13
        case 'Q':
            return 12
        case 'J':
            return 11
        case 'T':
            return 10
        case other:
            return int(card)


def highest_card(a: str, b: str) -> int:
    a_value = card_value(a)
    b_value = card_value(b)
    if a_value > b_value:
        return 1
    elif a_value < b_value:
        return -1
    return 0

def compare_hand_value(A: str, B: str) -> int:
    for a,b in zip(A, B):
        if a != b:
            return highest_card(a,b)
    return 0

def compare_type(a: str, b: str) -> int:
    a_type = get_type(a)
    b_type = get_type(b)
    if a_type > b_type:
        return 1
    elif a_type < b_type:
        return -1
    return 0

def compare_hands(a: Player, b: Player) -> int:
    return compare_type(a.hand,b.hand) or compare_hand_value(a.hand, b.hand)
    

data = read_file("day7/example.txt")
players = parse_players(data)
sorted_players = sorted(players, key=cmp_to_key(compare_hands))
bids = [rank * player.bid for rank, player in enumerate(sorted_players, 1)]
print(sum(bids))    

6440


### Part 2

In [20]:
def card_value(card: str):
    match card:
        case 'A':
            return 14
        case 'K':
            return 13
        case 'Q':
            return 12
        case 'J':
            return 1
        case 'T':
            return 10
        case other:
            return int(card)

def get_type(cards: str) -> Card_type:
    unique_labels = set(cards)

    wildcards = 0
    if "J" in unique_labels:
        unique_labels.remove("J")
        wildcards = cards.count("J")
    
    match len(unique_labels):
        case 0:
            return Card_type.FIVE_OF_A_KIND
        case 1:
            return Card_type.FIVE_OF_A_KIND
        case 2:
            sorted_cards = sorted([cards.count(c) for c in unique_labels], reverse=True)
            if (sorted_cards[0] + wildcards) == 4:
                return Card_type.FOUR_OF_A_KIND
            return Card_type.FULL_HOUSE
        case 3:
            sorted_cards = sorted([cards.count(c) for c in unique_labels], reverse=True)
            if (sorted_cards[0] + wildcards) == 3:
                return Card_type.THREE_OF_A_KIND
            return Card_type.TWO_PAIR

        case 4:
            return Card_type.PAIR
        case other:
            return Card_type.HIGH_CARD

data = read_file("day7/example.txt")
players = parse_players(data)
sorted_players = sorted(players, key=cmp_to_key(compare_hands))
bids = [rank * player.bid for rank, player in enumerate(sorted_players, 1)]
print(sum(bids))    

5905


# Day 8: Haunted Wasteland

### Part 1

In [21]:
@dataclass
class Node():
    label: str
    left: str
    right: str

def parse_data(lines: list[str]) -> (str, dict[str, Node]):
    tree = {}
    for line in lines[2:]:
        label, left, right = re.findall(r'(\w{3})',line)
        tree[label] = Node(label, left, right)

    return lines[0], tree

def start(tree: dict[str, Node]) -> Node:
    return tree['AAA']

def go_to_zzz(tree: dict[str, Node], instructions: str) -> int:
    current_node = start(tree)

    steps = 0
    while(current_node.label != 'ZZZ'):
        s = instructions[steps % len(instructions)]
        steps += 1
        current_node = tree[current_node.left] if s == "L" else tree[current_node.right]
    
    return steps

data = read_file("Day8/example1.txt")

instructions, tree = parse_data(data)
steps = go_to_zzz(tree, instructions)
print(steps)


2


### Part 2


This solution was my second attempt.
This solution finds first cycle and returns the steps to the last found Z. In my input the Z:s was right before the cycle wrapt around.

So I calculate the number of steps until I reach this Z for each A. Then I find the least common multiple between these steps.

I tried a multithreaded solution first, where I spawn a worker for each start, ran it for 10^6 steps and compared the output, but after looking for the answer for 60 min I gave up and tried this solution.


In [22]:
def starts(tree) -> list[Node]:
    return [node for node in tree.values() if node.label.endswith('A')]

def goal_reached(nodes: list[Node]) -> bool:
    return all([n.label.endswith("Z") for n in nodes]) 

def next_node(node: Node, tree: list[Node], instruction: str) -> Node:
    return tree[node.left] if instruction == "L" else tree[node.right]

def find_cycle(tree: dict[str, Node], start: Node, instructions: str) -> int:
    current_node = start
    visited = {}

    steps = 0
    while f"{steps % len(instructions)}_{current_node.label}" not in visited:
        k = f"{steps % len(instructions)}_{current_node.label}"
        visited[k] = steps
        s = instructions[steps % len(instructions)]
        steps += 1
        current_node = next_node(current_node, tree, s)
    

    zs = [n for n in visited.keys() if n.endswith("Z")]
    
    # My input only had 1 z in the entire cycle, and it was at step 0 modulos len of instructions.
    # The reason I return last z in list of zs is to make it work with the example that had 2 z in one cycle.
    # I.E this algorithm is not generic but works for the example and my input 
    return visited[zs[-1]]


def go_to_all_ending_with_z(tree: dict[str, Node], instructions: str) -> int:
    z_cycles = [find_cycle(tree, node, instructions) for node in starts(tree)]
    
    return math.lcm(*z_cycles)

data = read_file("Day8/example3.txt")
instructions, tree = parse_data(data)
steps = go_to_all_ending_with_z(tree, instructions)
print(steps)


6


# Day 9: Mirage Maintenance

In [23]:
def parse_sequence(line: str) -> list[int]:
    return [int(x) for x in line.split()]

def parse_sequences(lines: list[str]) -> list[list[int]]:
    return [parse_sequence(line) for line in lines]

def difference(sequence: list[int]) -> list[int]:
    return [sequence[i+1] - sequence[i] for i in range(len(sequence)-1)]

def predict_next(sequence: list[int]) -> int:
    ds = [difference(sequence)]
    
    while any(ds[-1]):
        d = difference(ds[-1])
        ds.append(d)

    last = [d[-1] for d in ds]
    return sum(last) + sequence[-1]

data = read_file("Day9/example.txt")
sequences = parse_sequences(data)

res = sum([predict_next(s) for s in sequences])
print(res)

114


### Part 2

I implemented a *predict backwards* function in this part, but Amanda showed me that I could just reverse the sequence and re-use my *predict next* from part 1, anyway, this works, yay!

In [24]:
from functools import reduce  # omit on Python 2

def predict_backwards(sequence: list[int]) -> int:
    ds = [difference(sequence)]
    
    while any(ds[-1]):
        d = difference(ds[-1])
        ds.append(d)

    first = [d[0] for d in ds[::-1]]
    first.append(sequence[0])

    return reduce(lambda x, y: y-x, first)

data = read_file("Day9/example.txt")
sequences = parse_sequences(data)

res = sum(([predict_backwards(s) for s in sequences]))
print(res)

2


# Day 10: Pipe Maze

In [113]:
@dataclass(frozen=True, slots=True)
class Position():
    row: int
    col: int

@dataclass
class Tile():
    pos: Position
    connections: list[Position]
    chr: str

# Pad the matrix with dots to avoid handling out of bounds issues
def pad(maze: list[str], chr='x') -> list[str]:
    rows = len(maze)
    y_pad = [chr * (rows + 2)]
    return y_pad + [chr + row + chr for row in maze] + y_pad

def get_chr(maze: list[str], position: Position) -> str:
    return maze[position.row][position.col]

def get_connections_offset(chr: str) -> list[Position]:
    match chr:
        case '|':
            return [Position(-1, 0), Position(1,0)]
        case '-':
            return [Position(0, -1), Position(0, 1)]
        case 'L':
            return [Position(-1, 0), Position(0,1)]
        case 'J':
            return [Position(-1, 0), Position(0,-1)]
        case '7':
            return [Position(1, 0), Position(0, -1)]
        case 'F':
            return [Position(1, 0), Position(0,1)]
        # case '.':
        #     return []
        case 'S':
            return [Position(1,0), Position(-1,0), Position(0,-1), Position(0,1)]
        case _:
            return []
        
def add(a: Position, b: Position) -> Position:
    return Position(a.row + b.row, a.col + b.col)
        
def get_tile(maze: list[str], position: Position):
    chr = get_chr(maze, position)
    return Tile(position, [add(position, offset) for offset in get_connections_offset(chr)], chr)

def find_start(lines: list[str]) -> Tile:
    for row, line in enumerate(lines[1:-2],1):
        for col, chr in enumerate(line[1:-2],1):
            if chr == 'S':
                pos = Position(row, col)
                return get_tile(lines, pos)

def next(t: Tile, prev: Position) -> Position | None:
    connection = [x for x in t.connections if x.row != prev.row or x.col != prev.col]
    return None if not len(connection) else connection.pop()

def loop_length(start: Tile, maze: list[str]) -> int:

    # Start with last connection (For some reason, I got the wrong count when starting with an other connection)
    # This was the right answer so I didn't debug more but coninued with part 2
    for connection_start in start.connections[::-1]:
        i = 1
        prev_pos = start.pos
        current_pos = connection_start
        while current_pos:
            current = get_tile(maze, current_pos)
            if current.chr == start.chr:
                return i

            i += 1
            temp = next(current, prev_pos)
            prev_pos = current_pos
            current_pos = temp
    
    return -1 # No connection is leading to start

data = read_file("day10/example1.txt")
padded = pad(data)

start = find_start(padded)
l = loop_length(start, padded)
print(int(l * 0.5))

4


# Part 2

This solution also contain a bug, as it don't handle all the example cases. But it works for some and for my puzzle input. 
Pherhaps I will revisit this in the future to fix the bugs (probably not)

In [151]:
def set_chr(maze: list[str], position: Position, chr: str):
    row = maze[position.row]
    new_row = row[:position.col] + chr + row[position.col + 1:]
    maze[position.row] = new_row

def print_debug(maze: list[str]):
    for l in maze:
        print(l)

def get_loop_map(start: Tile, maze: list[str]) -> list[str]:
    for connection_start in start.connections[::-1]:
        res = [' ' * len(row) for row in maze]
        set_chr(res, start.pos, "S")

        prev_pos = start.pos
        current_pos = connection_start
        while current_pos:
            current = get_tile(maze, current_pos)
            if current.chr == start.chr:
                start_delta_col = connection_start.col - start.pos.col  
                start_delta_row = connection_start.row - start.pos.row
                end_delta_col = prev_pos.col - start.pos.col
                end_delta_row = prev_pos.row - start.pos.row

                if start_delta_col == 1 and end_delta_row == 1:
                    set_chr(res, current.pos, "F")
                elif start_delta_col == -1 and end_delta_row == 1:
                    set_chr(res, current.pos, "7")
                elif abs(start_delta_row) and abs(end_delta_row):
                    set_chr(res, current.pos, "|")
                else:
                    # We dont need to count these edges in fill function
                    set_chr(res, current.pos, "S")
                return res

            set_chr(res, current.pos, current.chr)
            temp = next(current, prev_pos)
            prev_pos = current_pos
            current_pos = temp
    
        return res

def fill_polygon(maze: list[str]):
    row_end = len(maze) - 1 # padded
    col_end = len(maze[0]) -1 # padded

    for row in range(1, row_end):
        for col in range(1, col_end):
            if get_chr(maze, Position(row, col)) == " ":
                right_of = maze[row][col+1:col_end]
                intersections = right_of.count("|") + right_of.count("7") + right_of.count("F") # Count number of times we cross the polygon

                # if odd number of intersections, we are inside the polygon
                # https://www.geeksforgeeks.org/how-to-check-if-a-given-point-lies-inside-a-polygon/
                if intersections % 2 == 1:
                    set_chr(maze, Position(row,col), "I")

def count_i(maze: list[str]) -> int:
    return sum([row.count('I') for row in maze])

data = read_file("day10/example3.txt")
padded = pad(data)

start = find_start(padded)
loop_map = get_loop_map(start, padded)
fill_polygon(loop_map)
print_debug(loop_map)
res = count_i(loop_map)
print(res)
